# ADULT

In [1]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,  OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_selector as selector


In [2]:

def incomeFixer(x):
    if x == "<=50K":
        return 0
    else:
        return 1
def clean_adult(df):
    ad_var = ['age','workclass','fnlwgt','education',
    'education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','target']
    df.columns = ad_var
    copy=df.copy()
    copy = copy.replace({' ?': np.nan, 0: np.nan, 99999: np.nan})
    copy = copy.replace(" ","",regex=True)
    copy = copy.replace("-","",regex=True)
    # f.replace('_', '', regex=True)
    copy["target"] = copy.apply(lambda x: incomeFixer(x['target']), axis=1) 
    cp_dp = copy.copy()
    cp_dp = cp_dp.drop(columns=['education','fnlwgt','capital-gain','capital-loss'])
    DataFrame = cp_dp.copy()
    categorical_missing = ['workclass','occupation','native-country']
    for ColName in categorical_missing:
        most_frequent_category=DataFrame[ColName].mode()[0]
    # replace nan values with most occured category
        DataFrame[ColName + "_Imputed"] = DataFrame[ColName]
        DataFrame[ColName + "_Imputed"].fillna(most_frequent_category,inplace=True)
        DataFrame[ColName] = DataFrame[ColName + "_Imputed"]
        DataFrame = DataFrame.drop([ColName + "_Imputed"], axis = 1)
    return DataFrame


def random_adult():
    adult_raw = pd.read_csv('../data/raw/adult/adult.data', header=None)
    cleaning_adult = clean_adult(adult_raw)
    adult_train = cleaning_adult.sample(n=5000, replace=False)
    adult_test = cleaning_adult.drop(adult_train.index)
    adult_train.to_csv('../data/train/adult.csv', index=False)
    adult_test .to_csv('../data/test/adult.csv', index=False)
    adult_train=pd.read_csv('../data/train/adult.csv')
    adlt_test=pd.read_csv('../data/train/adult.csv')
    return [adult_train, adult_test]
    

In [21]:
! pwd

/Users/anthea-mac/Documents/FA20/COGS118A/Final Project/notebooks/temp


In [26]:
import sys
sys.path.insert(0,'data/')
!pwd
pd.read_csv('./test/adult.csv').head()

/Users/anthea-mac/Documents/FA20/COGS118A/Final Project/notebooks/temp


FileNotFoundError: [Errno 2] No such file or directory: './test/adult.csv'

# NURSERY

In [19]:
import json
with open("nsrorder.json", "r") as read_file:
    print("Converting JSON encoded data into Python dictionary")
    developer = json.load(read_file)
    print("Decoded JSON Data From File")
    for key, value in developer.items():
        print(key, ":", value)
    print("Done reading json file")
def check_item(raw, in_r):
    for i in in_r:
        if i in raw:
            pass
        else:
            print('wrong item is : ', i)
    #return'No Error Found in column ' 
def clean_nsr(df):
    od =developer
    nsr_var = ['parents', 'has_nurs', 'form', 'children', 'housing','finance','social', 'health','target']
    df.columns = nsr_var
    raw = df.copy()
    raw = raw.replace({'inconv':0, 'convenient':1})
    df = df.replace('_', '', regex=True)
    df = df.drop(columns = ['finance'])
    for i in df.columns:
        df[i] = df[i].astype('category')
        r = od[i]
        cat_r = CategoricalDtype(categories=r, ordered=True)
        # give the order
        df[i] = df[i].cat.reorder_categories(r, ordered=True)

    df['finance'] = raw['finance']
    return df
nsr_raw = pd.read_csv('../data/raw/nursey/nursery.data', header=None)
cleaning_nsr = clean_nsr(nsr_raw)
cleaning_nsr['target'] =cleaning_nsr['target'].replace( ['notrecom', 'recommend', 'veryrecom','priority', 'specprior'],[0,0,2,3,4] )
display(cleaning_nsr.head())
nsr_train = cleaning_nsr.sample(n=10000, replace=False)
nsr_test = cleaning_nsr.drop(nsr_train.index)
nsr_train.to_csv('../data/train/nsr.csv', index=False)
nsr_test .to_csv('../data/test/nsr.csv', index=False)


Converting JSON encoded data into Python dictionary
Decoded JSON Data From File
parents : ['usual', 'pretentious', 'greatpret']
has_nurs : ['verycrit', 'critical', 'improper', 'lessproper', 'proper']
form : ['foster', 'incomplete', 'complete', 'completed']
children : ['1', '2', '3', 'more']
housing : ['critical', 'lessconv', 'convenient']
social : ['problematic', 'slightlyprob', 'nonprob']
health : ['notrecom', 'priority', 'recommended']
target : ['notrecom', 'priority', 'specprior', 'recommend', 'veryrecom']
Done reading json file


,parents,has_nurs,form,children,housing,social,health,target,finance
0,usual,proper,complete,1,convenient,nonprob,recommended,0,1
1,usual,proper,complete,1,convenient,nonprob,priority,3,1
2,usual,proper,complete,1,convenient,nonprob,notrecom,0,1
3,usual,proper,complete,1,convenient,slightlyprob,recommended,0,1
4,usual,proper,complete,1,convenient,slightlyprob,priority,3,1


In [15]:
df = pd.read_csv('../data/test/nsr.csv')
y = df.target
X=df.drop(columns=['target'])
X.head()

,parents,has_nurs,form,children,housing,social,health,finance
0,usual,proper,complete,1,convenient,slightlyprob,recommended,1
1,usual,proper,complete,1,convenient,slightlyprob,notrecom,1
2,usual,proper,complete,1,convenient,problematic,priority,1
3,usual,proper,complete,1,convenient,problematic,notrecom,1
4,usual,proper,complete,1,convenient,nonprob,priority,0


In [18]:
df['target'].describe()

count    2960.000000
mean        2.299324
std         1.676105
min         0.000000
25%         0.000000
50%         3.000000
75%         4.000000
max         4.000000
Name: target, dtype: float64

In [11]:
preprocessor_ordinal = ColumnTransformer(transformers=[('categoricals', OrdinalEncoder(),
                                                selector(dtype_include=["object", "category"])),
                                               ('numericals', StandardScaler(), selector(
                                                   dtype_include=["int", 'float']))
                                               ])

In [12]:
df

,parents,has_nurs,form,children,housing,social,health,target,finance
0,usual,proper,complete,1,convenient,nonprob,notrecom,0,1
1,usual,proper,complete,1,convenient,slightlyprob,recommended,1,1
2,usual,proper,complete,1,convenient,slightlyprob,priority,3,1
3,usual,proper,complete,1,convenient,slightlyprob,notrecom,0,1
4,usual,proper,complete,1,convenient,problematic,recommended,3,1


# AVL

In [35]:

avl=pd.read_csv('../data/raw/avl_set/avila-tr.txt', header=None)
avl.columns = avl.columns.astype(str)
avl=avl.rename(columns={'10':'target'})
avl_train = avl.sample(n=5000, replace=False)
avl_test = cleaning_nsr.drop(avl_train.index)
nsr_train.to_csv('../data/train/avl.csv', index=False)
nsr_test .to_csv('../data/test/avl.csv', index=False)
print(avl.info())
avl.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10430 entries, 0 to 10429
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       10430 non-null  float64
 1   1       10430 non-null  float64
 2   2       10430 non-null  float64
 3   3       10430 non-null  float64
 4   4       10430 non-null  float64
 5   5       10430 non-null  float64
 6   6       10430 non-null  float64
 7   7       10430 non-null  float64
 8   8       10430 non-null  float64
 9   9       10430 non-null  float64
 10  target  10430 non-null  object 
dtypes: float64(10), object(1)
memory usage: 896.5+ KB
None


,0,1,2,3,4,5,6,7,8,9,target
0,0.266074,-0.165620,0.320980,0.483299,0.172340,0.273364,0.371178,0.929823,0.251173,0.159345,A
1,0.130292,0.870736,-3.210528,0.062493,0.261718,1.436060,1.465940,0.636203,0.282354,0.515587,A
2,-0.116585,0.069915,0.068476,-0.783147,0.261718,0.439463,-0.081827,-0.888236,-0.123005,0.582939,A
3,0.031541,0.297600,-3.210528,-0.583590,-0.721442,-0.307984,0.710932,1.051693,0.594169,-0.533994,A
4,0.229043,0.807926,-0.052442,0.082634,0.261718,0.148790,0.635431,0.051062,0.032902,-0.086652,F


# POWER

In [38]:
from supervised.automl import AutoML
from sklearn.model_selection import train_test_split


In [ ]:
from sklearn.metrics import accuracy_score
prediction = automl.predict(X_test)

In [8]:
dic={}
dic['1']=4
dic['2'] =2
dic['3']=1
dic

{'1': 4, '2': 2, '3': 1}

In [19]:
dic.keys()
dic.values()

dict_values([4, 2, 1])

In [35]:
list(dic)

['1', '2', '3']